In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from tqdm import tqdm
from collections import Counter

In [12]:
n_case = 21
n_control = 23

df = pd.read_csv('DE/norm_camp.tsv', sep='\t')
df

,gene_symbol,control,control.1,control.2,control.3,control.4,control.5,control.6,control.7,control.8,...,case.11,case.12,case.13,case.14,case.15,case.16,case.17,case.18,case.19,case.20
0,LIG3,8.402037,8.196623,8.511670,8.642240,7.876629,8.447436,8.443265,8.388793,8.459005,...,8.065102,8.152883,8.054064,7.293737,7.825344,7.793442,8.128742,7.766462,8.305929,7.629954
1,FARP2,7.178840,7.560245,7.287791,7.895216,7.339165,7.965488,7.618820,8.232080,7.521733,...,6.817028,7.153652,6.850390,6.578172,7.454288,7.430140,7.341216,6.948069,7.205742,7.301272
2,MMP25,13.217405,14.407236,11.729645,12.727599,14.051578,12.588177,13.384104,13.660015,12.899144,...,14.921608,14.915686,14.492913,15.700787,14.359867,14.168115,13.714812,14.659903,13.663062,14.307371
3,SERPINB1,11.484395,11.731361,11.070671,11.074353,12.088068,11.077276,11.145356,11.875659,11.008006,...,12.212213,12.405420,12.262497,12.943465,11.803746,11.642090,11.282862,13.245604,11.351687,13.018834
4,PRKCH,10.286177,9.870643,10.613959,10.397713,9.890621,10.477413,10.458713,10.503818,10.578871,...,9.753053,9.718684,10.012889,9.408923,9.943304,10.205771,9.658076,9.650722,9.928016,9.469700
5,SLC4A8,6.573237,6.631800,6.713772,6.617472,6.379827,6.520811,6.348942,5.984036,6.413820,...,5.780421,5.834269,6.433541,5.312622,5.921761,6.395760,5.921598,5.318794,6.127855,6.147029
6,SLC2A3,11.519591,11.813977,10.490574,10.433349,12.433086,10.744125,11.075684,11.767405,11.225530,...,12.318164,12.553408,12.455251,13.115201,12.084355,12.249457,10.710742,13.705027,11.480875,12.986218
7,MYO9A,7.948771,6.804673,7.964991,7.983743,7.433705,8.175986,7.988898,7.566911,7.799153,...,6.725045,7.170320,7.020818,6.767891,7.044872,7.224266,6.841745,7.117134,6.963337,7.001364
8,IKBKAP,9.135031,8.425429,9.077440,9.257932,8.792059,9.259954,9.242665,9.032992,9.342378,...,8.102092,8.624132,8.834665,8.096805,8.518256,8.585264,8.438491,8.368901,8.826869,8.508431
9,OSBPL3,8.610284,7.635362,9.476768,8.439343,7.909890,8.772866,8.239456,8.563406,8.349922,...,7.623966,7.461879,7.563428,7.362551,7.742666,8.064485,7.653970,7.707176,7.524648,7.742964


In [17]:
genes = df['gene_symbol']
df.drop('gene_symbol', axis=1, inplace=True)

column_names = list(df)

control_names = column_names[:n_control]
case_names = column_names[n_control:]

ill = Counter()
healthy = Counter()

In [3]:
def t(ser):
    n_case = 21
    n_control = 23
    
    controls = ser[control_names]

    cases = ser[case_names]

    xA = cases.mean()
    xB = controls.mean()

    sA = cases.var()
    sB = controls.var()

    spooled = ((n_case - 1)*sA + (n_control - 1)*sB) / (n_case + n_control - 2)
    if spooled == 0:
        if xA == xB:
            return 0
        return float('inf')
    
    stat = abs(xA - xB) / (sqrt(spooled)*sqrt(1/n_case + 1/n_control))
    return stat

In [22]:
for i in tqdm(range(len(df))):
    ser = df.loc[i]
    
    controls = ser[control_names]
    cases = ser[case_names]
    xA = cases.mean()
    xB = controls.mean()

    stat = t(ser)
    pcount = 0
    for _ in range(100):
        rand_ser = pd.Series(np.random.permutation(ser), index=column_names)
        rand_t = t(rand_ser)
        pcount += (rand_t >= stat)
    p = pcount / 100
    if p <= 0.05:
        r = xA - xB
        expr = abs(r)
        if r < 0:
            healthy[genes[i]] = expr
        else:
            ill[genes[i]] = expr

100%|██████████| 300/300 [00:43<00:00,  7.03it/s]


In [28]:
with open('ill.tsv', 'w') as file:
    for gene, i in ill.most_common():
        file.write(f'{gene}\t{i}\n')

In [24]:
with open('healthy.tsv', 'w') as file:
    for gene, i in healthy.most_common():
        file.write(f'{gene}\t{i}\n')

In [26]:
len(healthy)

109

In [30]:
True or False and True

True